
# MI alapú alkalmazásfejlesztés

<a target="_blank" href="https://colab.research.google.com/github/reguly/aiappdev/blob/main/Crew.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
!pip install crewai
!pip install crewai_tools

In [12]:
import os
os.environ['OPENAI_API_KEY'] = '' #!!!! itt kell megadni az API kulcsot

In [22]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

## Define Agents
spamfilter = Agent(
  role='spamfilter',
  goal='''Decide whether a text is spam or not.''',
  backstory='You are an expert spam filter with years of experience. You DETEST advertisements, newsletters and vulgar language.',
  llm=llm,
  verbose=True,
  allow_delegation=False
)

analyst = Agent(
  role='analyse',
  goal='''You will distill all arguments from all discussion members. Identify who said what. You can reword what they said as long as the main discussion points remain.''',
  backstory='You are an expert discussion analyst.',
  llm=llm,
  verbose=True,
  allow_delegation=False
)

scriptwriter = Agent(
  role='scriptwriter',
  goal='Turn a conversation into a movie script. Only write the dialogue parts. Do not start the sentence with an action. Do not specify situational descriptions. Do not write parentheticals.',
  backstory='''You are an expert on writing natural sounding movie script dialogues. You only focus on the text part and you HATE directional notes.''',
  llm=llm,
  verbose=True,
  allow_delegation=False

)

formatter = Agent(
  role='formatter',
  goal='''Format the text as asked. Leave out actions from discussion members that happen between brackets, eg (smiling).''',
  backstory='You are an expert text formatter.',
  llm=llm,
  verbose=True,
  allow_delegation=False
)

scorer = Agent(
  role='scorer',
  goal='''You score a dialogue assessing various aspects of the exchange between the participants using a 1-10 scale, where 1 is the lowest performance and 10 is the highest:
  Scale:
  1-3: Poor - The dialogue has significant issues that prevent effective communication.
  4-6: Average - The dialogue has some good points but also has notable weaknesses.
  7-9: Good - The dialogue is mostly effective with minor issues.
  10: Excellent - The dialogue is exemplary in achieving its purpose with no apparent issues.
  Factors to Consider:
  Clarity: How clear is the exchange? Are the statements and responses easy to understand?
  Relevance: Do the responses stay on topic and contribute to the conversation's purpose?
  Conciseness: Is the dialogue free of unnecessary information or redundancy?
  Politeness: Are the participants respectful and considerate in their interaction?
  Engagement: Do the participants seem interested and actively involved in the dialogue?
  Flow: Is there a natural progression of ideas and responses? Are there awkward pauses or interruptions?
  Coherence: Does the dialogue make logical sense as a whole?
  Responsiveness: Do the participants address each other's points adequately?
  Language Use: Is the grammar, vocabulary, and syntax appropriate for the context of the dialogue?
  Emotional Intelligence: Are the participants aware of and sensitive to the emotional tone of the dialogue?
  ''',
  backstory='You are an expert at scoring conversations on a scale of 1 to 10.',
  llm=llm,
  verbose=True,
  allow_delegation=False
)

In [23]:
discussion = '''From: Hospital Management <management@hospital.com>
To: All Staff <allstaff@hospital.com>
Subject: Urgent: Maintenance of Restroom Facilities

Dear Team,

It has come to our attention that the condition of the restroom facilities throughout our hospital has recently been suboptimal. We would like to remind everyone of the importance of maintaining a clean and hygienic environment, not only for ourselves but also for our patients and visitors.

Please ensure that you are cleaning up after yourselves and disposing of waste appropriately. Your cooperation is essential in keeping our hospital a safe and welcoming place for everyone.

Thank you for your attention to this matter.

Best Regards,
Hospital Management

From: Jane Doe, RN <jdoe@hospital.com>
To: Hospital Management <management@hospital.com>
CC: All Staff <allstaff@hospital.com>
Subject: Re: Urgent: Maintenance of Restroom Facilities

Dear Management,

While I understand the importance of cleanliness, I must point out that the restrooms often lack basic supplies such as toilet paper and soap, making it difficult to maintain hygiene standards. Furthermore, the overall working conditions, including long hours and insufficient equipment, significantly affect our ability to perform our duties effectively.

I hope these issues can be addressed promptly.

Regards,
Jane Doe, RN

From: Hospital Management <management@hospital.com>
To: All Staff <allstaff@hospital.com>
Subject: Re: Concerns Raised About Restroom Facilities and Working Conditions

Dear Staff,

Thank you for your feedback. We are currently addressing the supply shortages in our restrooms and are working diligently to ensure that all facilities are appropriately stocked. We appreciate your hard work and understand that conditions have been challenging.

We are committed to improving your work environment and ask for your patience as we resolve these issues. Your dedication to our patients and each other is greatly valued.

Warm regards,
Hospital Management

From: Tom Brown, Technician <tbrown@hospital.com>
To: Hospital Management <management@hospital.com>
CC: All Staff <allstaff@hospital.com>
Subject: Re: Concerns Raised About Restroom Facilities and Working Conditions

Dear Management,

The response to our concerns, while appreciated, does not fully address the chronic understaffing and lack of maintenance support that have been ongoing issues. The restrooms are a symptom of larger systemic problems that require more than just restocking supplies. We need actionable changes to improve not only the facilities but also our working conditions.

Sincerely,
Tom Brown

From: Emily White, Pharmacist <ewhite@hospital.com>
To: All Staff <allstaff@hospital.com>
Subject: Suggestion for Improvement

Hi Everyone,

To help address some of our immediate concerns, I propose setting up a staff-led committee to monitor and report issues with facilities and supplies directly to management. This could expedite resolutions and improve communication. We could rotate the responsibility monthly to ensure it doesn't become a burden on any one individual.

Would love to hear your thoughts on this or any other suggestions!

Best,
Emily White

From: John Smith, Lab Technician <jsmith@hospital.com>
To: All Staff <allstaff@hospital.com>
Subject: Re: Suggestion for Improvement

Hi Emily,

Great idea! I also suggest we could use a digital checklist for the restroom facilities that all employees can access and update in real time. This would make it easier to keep track of what's missing and what needs immediate attention.

Cheers,
John Smith

From: Hospital Management <management@hospital.com>
To: All Staff <allstaff@hospital.com>
Subject: Re: Suggestion for Improvement

Dear Team,

We are inspired by your proactive approach and are pleased to announce that we will be implementing the staff-led committee idea suggested by Emily White. Furthermore, in recognition of her constructive initiative, Emily has been elected to join the Board of Directors to represent staff interests and improve our hospital’s operations.

Thank you all for your valuable suggestions and engagement.

Best Regards,
Hospital Management
'''

In [ ]:
# Filter out spam and vulgar posts
task0 = Task(description='Read the following newsgroup post. If this contains vulgar language reply with STOP . If this is spam reply with STOP.\n### NEWGROUP POST:\n' + discussion, 
             agent=spamfilter,
            expected_output='STOP if the post contains spam or is vulgar. GO otherwise')
result = task0.execute_sync()
if "STOP" in result:
    #stop here and proceed to next post
    print('This spam message will be filtered out')
else:
    print(result)

In [ ]:
# process post with a crew of agents, ultimately delivering a well formatted dialogue
task1 = Task(description='Analyse in much detail the following discussion:\n### DISCUSSION:\n' + discussion, 
             agent=analyst,
             expected_output='A distilled summary of the discussion'
            )
task2 = Task(description='Create a dialogue heavy screenplay from the discussion, between two persons. Do NOT write parentheticals. Leave out wrylies. You MUST SKIP directional notes.',
             agent=scriptwriter,
             expected_output='The text of the screenplay'
            )
task3 = Task(description='''Format the script exactly like this:
  ## (person 1):
  (first text line from person 1)
             
  ## (person 2):
  (first text line from person 2)
             
  ## (person 1):
  (second text line from person 1)
             
  ## (person 2):
  (second text line from person 2)
  
  ''',
             agent=formatter,
             expected_output='The formatted script')
crew = Crew(
  agents=[analyst, scriptwriter,formatter],
  tasks=[task1, task2, task3],
  verbose=True,
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)
result = crew.kickoff()

In [ ]:
news = '''
UK and South Korea to co-host AI Seoul Summit

The UK and South Korea are set to co-host the AI Seoul Summit on the 21st and 22nd of May. This summit aims to pave the way for the safe development of AI technologies, drawing on the cooperative framework laid down by the Bletchley Declaration.

The two-day event will feature a...

12 April 2024 | Artificial Intelligence
----------------------------------
US and Japan announce sweeping AI and tech collaboration

The US and Japan have unveiled a raft of new AI, quantum computing, semiconductors, and other critical technology initiatives.

The ambitious plans were announced this week by President Biden and Japanese Prime Minister Kishida Fumio following Kishida's Official Visit to the White...

11 April 2024 | Artificial Intelligence
----------------------------------
OpenAI makes GPT-4 Turbo with Vision API generally available

OpenAI has announced that its powerful GPT-4 Turbo with Vision model is now generally available through the company's API, opening up new opportunities for enterprises and developers to integrate advanced language and vision capabilities into their applications.

The launch of GPT-4...

10 April 2024 | Artificial Intelligence
----------------------------------
Stability AI unveils 12B parameter Stable LM 2 model and updated 1.6B variant

Stability AI has introduced the latest additions to its Stable LM 2 language model series: a 12 billion parameter base model and an instruction-tuned variant. These models were trained on an impressive two trillion tokens across seven languages: English, Spanish, German, Italian, French,...

9 April 2024 | Artificial Intelligence
----------------------------------
Microsoft AI opens London hub to access ‘enormous pool’ of talent

Microsoft is doubling down on its AI efforts in the UK with the opening of a major new AI hub in London. The Microsoft AI London outpost will focus on advancing state-of-the-art language models, supporting infrastructure, and tooling for foundation models.

The hub will be led by AI...

8 April 2024 | Artificial Intelligence
----------------------------------
ML Olympiad returns with over 20 challenges

The popular ML Olympiad is back for its third round with over 20 community-hosted machine learning competitions on Kaggle.

The ML Olympiad – organised by groups including ML GDE, TFUG, and other ML communities – aims to provide developers with hands-on opportunities to learn and...

8 April 2024 | Applications
----------------------------------
Microsoft: China plans to disrupt elections with AI-generated disinformation

Beijing is expected to ramp up sophisticated AI-generated disinformation campaigns to influence several high-profile elections in 2024, according to Microsoft's threat intelligence team.

Microsoft warned that state-backed Chinese cyber groups – with assistance from North Korean...

5 April 2024 | Artificial Intelligence
----------------------------------
Why data quality is critical for marketing in the age of GenAI

A recent survey reveals that CMOs around the world are optimistic and confident about GenAI’s future ability to enhance productivity and create competitive advantage. Seventy per cent are already using GenAI and 19 per cent are testing it. And the main areas they’re exploring are...

4 April 2024 | Applications
----------------------------------
How to safeguard your business from AI-generated deepfakes 

Recently, cybercriminals used ‘deepfake’ videos of the executives of a multinational company to convince the company’s Hong Kong-based employees to wire out US $25.6 million. Based on a video conference call featuring multiple deepfakes, the employees believed that their UK-based chief...

3 April 2024 | Artificial Intelligence
----------------------------------
UK and US sign pact to develop AI safety tests

The UK and US have signed a landmark agreement to collaborate on developing rigorous testing for advanced AI systems, representing a major step forward in ensuring their safe deployments.

The Memorandum of Understanding – signed Monday by UK Technology Secretary Michelle Donelan and...

2 April 2024 | Artificial Intelligence
----------------------------------
Databricks claims DBRX sets ‘a new standard’ for open-source LLMs

Databricks has announced the launch of DBRX, a powerful new open-source large language model that it claims sets a new bar for open models by outperforming established options like GPT-3.5 on industry benchmarks. 

The company says the 132 billion parameter DBRX model surpasses...
28 March 2024 | Artificial Intelligence
'''

from crewai import Agent, Task, Crew

research_agent = Agent(
  role='Researcher',
  goal='Find and summarize the latest AI news',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  verbose=True,
  allow_delegation=False
)

task = Task(
  description='Find and summarize the latest AI news. The news are as follows: '+news,
  expected_output='A bullet list summary of the top 5 most important AI news',
  agent=research_agent
)

crew = Crew(
    agents=[research_agent],
    tasks=[task],
    verbose=True
)

result = crew.kickoff()
print(result)


In [31]:
os.environ['SERPER_API_KEY']='...' #SERPER key comes here!

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

research_agent = Agent(
  role='Researcher',
  goal='Find and summarize the latest AI news',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  verbose=True,
  allow_delegation=False
)

search_tool = SerperDevTool()

task = Task(
  description='Find and summarize the latest AI news',
  expected_output='A bullet list summary of the top 5 most important AI news',
  agent=research_agent,
  tools=[search_tool]
)

crew = Crew(
    agents=[research_agent],
    tasks=[task],
    verbose=True
)

result = crew.kickoff()
print(result)

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
#from langchain_community.llms import Ollama
#ollama = Ollama(model="command-r", base_url="http://localhost:11434")
from langchain_openai import ChatOpenAI
ollama = ChatOpenAI(model='gpt-4o-mini')

research_agent = Agent(
  role='Researcher',
  goal='Find and summarize the latest AI news',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  verbose=True,
  llm=ollama,
  function_calling_llm=ollama
)

search_tool = SerperDevTool()

task = Task(
  description='Find and summarize the latest AI news',
  expected_output='A bullet list summary of the top 5 most important AI news',
  agent=research_agent,
  tools=[search_tool]
)

crew = Crew(
    agents=[research_agent],
    tasks=[task],
    verbose=True
)

result = crew.kickoff()
print(result)